In [7]:
import numpy as np
import pandas as pd

# Прямая задача

In [8]:
def simplex(A, b, c):
    n, m = A.shape

    table = np.zeros((n, m + n), dtype=float)

    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]

    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    c_old = c.copy()

    c = np.zeros(n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]

    Cb = np.zeros(n, dtype=float)
    Bp = np.array([i for i in range(m, n + m)], dtype=float)  

    result = np.zeros(n + m)

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_max = {np.round(F, 2)}')

        k = np.argmax(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, n + m + 3:n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] > 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(result, 2)

        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        result[k] = fractions[r] * deltas[k] / c[k]

        table_old = table.copy()

        for i in range(n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('=' * 100)

In [9]:
c = np.array([4, 5], dtype=float) 

A = np.array([
    [2, 4],
    [1, 1],
    [2, 1]
], dtype=float)

b = np.array([560, 170, 300], dtype=float)

F_max, result = simplex(A, b, c)
print(f'F_max = {F_max}\nОптимальные значения переменных: \n{result}')

Промежуточный F_max = 0.0
Δ = [4. 5. 0. 0. 0.]
simpex table: 
     0    1      2    3    4    5    6    7      8
0  2.0  0.0  560.0  2.0  4.0  1.0  0.0  0.0  140.0
1  3.0  0.0  170.0  1.0  1.0  0.0  1.0  0.0  170.0
2  4.0  0.0  300.0  2.0  1.0  0.0  0.0  1.0  300.0
Промежуточный F_max = 700.0
Δ = [ 1.5   0.   -1.25  0.    0.  ]
simpex table: 
     0    1      2    3    4     5    6    7       8
0  1.0  5.0  140.0  0.5  1.0  0.25  0.0  0.0  280.00
1  3.0  0.0   30.0  0.5  0.0 -0.25  1.0  0.0   60.00
2  4.0  0.0  160.0  1.5  0.0 -0.25  0.0  1.0  106.67
Промежуточный F_max = 790.0
Δ = [ 0.   0.  -0.5 -3.   0. ]
F_max = 790.0
Оптимальные значения переменных: 
[ 22.5 140.    0.    0.    0. ]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10220\624538214.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


# Двойственная задача

In [10]:
def dual_simplex(A, b, c, M):
    
    def get_optimal_values(Bp, b, n, m):
        optimal_values = np.zeros(2 * n + m)
        for i in range(len(Bp)):
            if Bp[i] < len(optimal_values):
                optimal_values[int(Bp[i])] = b[i]
        return optimal_values

    n, m = A.shape

    table = np.zeros((n, m + 2 * n), dtype=float)

    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]

    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = -1
            else:
                table[j, i] = 0

    for i in range(n + m, 2 * n + m):
        for j in range(n):
            if i - m - n == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    c_old = c.copy()

    c = np.zeros(2 * n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]
    for i in range(n + m, 2 * n + m):
        c[i] = M

    Cb = np.array([M for _ in range(n)], dtype=float)
    Bp = np.array([i for i in range(m + n, m + 2 * n)], dtype=float) 

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + 2 * n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_min = {np.round(F, 2)}')

        k = np.argmin(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, 2 * n + m + 3: 2 * n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] < 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(get_optimal_values(Bp, b, n, m), 2)
        
        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        table_old = table.copy()

        for i in range(2 * n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('=' * 100)

In [11]:
M = 10 ** 3

c = np.array([560, 170, 300], dtype=float)

A = np.array([
    [2, 1, 2],
    [4, 1, 1]
], dtype=float)

b = np.array([4, 5], dtype=float)

In [12]:
F_min, result = dual_simplex(A, b, c, M=M)
print(f'F_min = {F_min}')
print(f'Оптимальное значение переменных:\n{result}')

Промежуточный F_min = 9000.0
Δ = [-5440. -1830. -2700.  1000.  1000.]
simpex table: 
    0       1    2    3    4    5    6    7    8    9     10
0  5.0  1000.0  4.0  2.0  1.0  2.0 -1.0  0.0  1.0  0.0  2.00
1  6.0  1000.0  5.0  4.0  1.0  1.0  0.0 -1.0  0.0  1.0  1.25
Промежуточный F_min = 2200.0
Δ = [    0.  -470. -1340.  1000.  -360.]
simpex table: 
    0       1     2    3     4     5    6     7    8     9    10
0  5.0  1000.0  1.50  0.0  0.50  1.50 -1.0  0.50  1.0 -0.50  1.0
1  0.0   560.0  1.25  1.0  0.25  0.25  0.0 -0.25  0.0  0.25  5.0
Промежуточный F_min = 860.0
Δ = [  0.   -23.33   0.   106.67  86.67]
simpex table: 
    0      1    2    3     4    5     6     7     8     9    10
0  2.0  300.0  1.0  0.0  0.33  1.0 -0.67  0.33  0.67 -0.33  3.0
1  0.0  560.0  1.0  1.0  0.17  0.0  0.17 -0.33 -0.17  0.33  6.0
Промежуточный F_min = 790.0
Δ = [  0.   0.  70.  60. 110.]
F_min = 790.0
Оптимальное значение переменных:
[0.5 3.  0.  0.  0.  0.  0. ]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10220\3750567698.py:60: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


Среди $\Delta_i$ не осталось отрицательных $\implies$ решение оптимально

Оптимальные значение переменных искусственного базиса $=0\implies$ решение допустимо